In [ ]:
# Пути к файлам
labels_csv='../input/dog-breed-identification/labels.csv'
sample_submission_csv='../input/dog-breed-identification/sample_submission.csv'

# Пути до картинок
jpg_train='../input/dog-breed-identification/train/{}.jpg'  
jpg_test='../input/dog-breed-identification/test/{}.jpg'

# Настройки нейросети
img_size = 224 # Размер изображения
num_class = 120 # Кол-во классов
batch_size = 32
Epochs = 64

# Настройки нейросети
#img_size = 224 # Размер изображения

## Загружаем библиотеки

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow.keras as keras
from keras import regularizers
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.metrics import categorical_accuracy, categorical_crossentropy
import os
import cv2, random, time, shutil, csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import time
from keras.utils import to_categorical
import gc
import tqdm

In [ ]:
def gen_graph(history, title):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('crossentropy ' + title)
    plt.ylabel('crossentropy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('categorical_accuracy ' + title)
    plt.ylabel('categorical_accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

## Считываем изображения и переводим в массив

In [ ]:
# set image size here
data_dir = '../input/dog-breed-identification'
data_df = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
class_names = sorted(data_df['breed'].unique())
print(f"No. of classes read - {len(class_names)}")
time.sleep(1)

images_list = sorted(os.listdir(os.path.join(data_dir, 'train')))
X = []
Y = []
i = 0
for image in tqdm.tqdm(images_list):
    cls_name = data_df[data_df['id'] == image[:-4]].iloc[0,1]
    cls_index = int(class_names.index(cls_name)) 

    # Reading RGB Images
    image_path = os.path.join(data_dir, 'train',image)
    orig_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    res_image = cv2.resize(orig_image,(img_size, img_size))
    X.append(res_image)
    Y.append(cls_index)
    i+=1

In [ ]:
# Converting to arrays
print(len(X), len(Y))
Xarr = np.array(X)
Yarr = np.array(Y).reshape(-1,1)

# converting labels to one hot
Yarr_hot = to_categorical(Y)

del(X)
print(Xarr.shape, Yarr.shape, Yarr_hot.shape)
gc.collect()

## Формируем датасет для обучения

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(Xarr, Yarr_hot, shuffle=True,  test_size=0.2)

In [ ]:
del Xarr, Yarr_hot, Yarr

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_valid.shape,Y_valid.shape)

## Создаем генератор изображений

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                            rotation_range=35, #поворот
                            horizontal_flip=True,
                            vertical_flip=True,
                            shear_range=15
                                )

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen.fit(X_train)
test_datagen.fit(X_valid)

# Создаем генераторы 
train_generator =train_datagen.flow(X_train, Y_train, 
                               batch_size=batch_size)

test_generator =test_datagen.flow(X_valid, Y_valid,
                              batch_size=batch_size)



In [ ]:
x,y = next(train_generator)

print(type(x))
print(train_generator.n)
print(x.shape)
print(y.shape)
del x,y

## Создаем модель нейросети

In [ ]:

# Создаем последовательную модель
model = Sequential()

# сверточный слой
model.add(Conv2D(64, (3, 3), padding='same',input_shape=(img_size, img_size, 3), activation='relu'))
# сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'))
# Слой регуляризации Dropout
model.add(Dropout(0.20))


# сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# сверточный слой
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'))
# Слой регуляризации Dropout
model.add(Dropout(0.20))


# сверточный слой
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
# сверточный слой
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# сверточный слой
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
# Третий слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'))
# Слой регуляризации Dropout
model.add(Dropout(0.20))

# сверточный слой
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
# сверточный слой
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# сверточный слой
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# Четвертый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'))
# Слой регуляризации Dropout
model.add(Dropout(0.20))

# сверточный слой
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
# сверточный слой
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# сверточный слой
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
# Четвертый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),padding='same'))
# Слой регуляризации Dropout
model.add(Dropout(0.20))





# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.30))
model.add(Dense(1024, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.30))


# Выходной полносвязный слой
model.add(Dense(num_class, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
          loss='categorical_crossentropy', 
           metrics=['categorical_accuracy'])

In [ ]:
print(model.summary())

Функция для остановки сети и сохранения лучших результатов

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop=EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

checkpoint_callback = ModelCheckpoint('model_best.hdf5',
                                      monitor='val_categorical_accuracy', 
                                      #save_best_only=True
                                     mode='auto')

## Обучаем нейросеть

In [ ]:
history = model.fit(
    train_generator,
    callbacks=[earlystop, checkpoint_callback],
    epochs=Epochs,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.n // test_generator.batch_size
    )

## Построим графики процесса обучения

In [ ]:
#График точности на валидационной и обучающей выборке
gen_graph(history,
          "график точности")

## Выберем лучшую модель и получим результаты для тестов

In [ ]:
del X_train, X_valid, Y_train, Y_valid

In [ ]:
images_list = sorted(os.listdir(os.path.join(data_dir, 'test')))
x_test=[]
for image in tqdm.tqdm(images_list):    
    image_path = os.path.join(data_dir, 'test',image)
    orig_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    res_image = cv2.resize(orig_image,(img_size, img_size))
    x_test.append(res_image)
x_test=np.array(x_test)

In [ ]:
x_test.shape

In [ ]:
from tensorflow.keras.models import load_model
model=load_model("model_best.hdf5")

In [ ]:
preds = model.predict(x_test)

In [ ]:
df_train = pd.read_csv(labels_csv)
df_test = pd.read_csv(sample_submission_csv)
labels = df_train['breed']
one_hot = pd.get_dummies(labels, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)

In [ ]:
sub.to_csv("output_rmsprop_aug.csv", index=False)